In [6]:
from gqn import GenerativeQueryNetwork, partition, Annealer
from dataset import GQN_Dataset
from torch.distributions import Normal
import shutil
import os
import numpy as np
import cv2
import torch
import glob

%matplotlib inline
from IPython.display import display, HTML
import matplotlib.pyplot as plt

In [7]:
device = torch.device("cpu")
model = GenerativeQueryNetwork(x_dim=3, v_dim=7, r_dim=256, h_dim=128, z_dim=64, L=8, dgf_dim=256).to(device)
models = glob.glob("checkpoints/checkpoint_model_*.pth")
models.sort(key=lambda x: os.path.getmtime(x))
checkpoint = torch.load(models[-1])
model.load_state_dict(checkpoint)

In [9]:
valid_dataset = GQN_Dataset(root_dir="/media/data/gqn-dataset/rooms-ring-camera/", train=False)
x, v = valid_dataset[-1]
x = x.view((1, *x.shape))
v = v.view((1, *v.shape))

max_m=5
x, v, x_q, v_q = partition(x, v, max_m, 5)
batch, n_context, *_ = x.shape

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/gqn-dataset/rooms-ring-camera/test'

In [ ]:
i=7
x.shape
for j in range(n_context):
    _img = x[i, j].numpy()
    _img = np.moveaxis(_img, [0, 1, 2], [-1, 0, 1])
    plt.subplot(2, n_context, j+1)
    plt.imshow(_img)
plt.show()

In [ ]:
y = x[i]
y = y.view((1, *y.shape))
y = y.repeat(36, 1, 1, 1, 1)
y_v = v[i]
y_v = y_v.view((1, *y_v.shape))
y_v = y_v.repeat(36, 1, 1)
coord1, coord2, coord3, _, _, p1, p2 = v_q[i]
#x, y, z, torch.cos(y), torch.sin(y), torch.cos(p), torch.sin(p)
for j in range(36):
    v_q[j] = torch.FloatTensor([coord1, coord2, coord3, np.cos(0.2*j), np.sin(0.2*j), p1, p2])

print(y.shape)
print(y_v.shape)
print(v_q.shape)

In [ ]:
x_mu = model.sample(y, y_v, v_q)

In [ ]:
for j in range(36):
    _img = x_mu[j].detach().numpy()
    _img = np.moveaxis(_img, [0, 1, 2], [-1, 0, 1])
#     plt.figure(figsize=(10,10))
    plt.imshow(_img)
    plt.show()

In [ ]:
print(x_mu.shape)

In [ ]:
i=4
j=1
for j in range(n_context):
    _img = y[i, j].numpy().astype(np.uint8)
    _img = np.moveaxis(_img, [0, 1, 2], [-1, 0, 1])
    plt.subplot(2, n_context, j+1)
    plt.imshow(_img)
plt.show()